In [ ]:
!nvidia-smi

Mon Nov 21 13:46:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Training

!mkdir /content/train
%cd /content/train

!wget -q https://cmu.box.com/shared/static/jg6qmgmh8zmunlpyj4sm5nfw9nj64cup --content-disposition --show-progress
!wget -q https://cmu.box.com/shared/static/c3e3caenl1zis2cu8beukh70jc1evv6f --content-disposition --show-progress

!tar -xf clean.tar
!tar -xf noisy.tar

# Testing

!mkdir /content/test
%cd /content/test

!wget -q https://cmu.box.com/shared/static/6bs853ij1arcjxfd4v77es9xbf7hmsgb --content-disposition --show-progress
!wget -q https://cmu.box.com/shared/static/kpg2bbz6u9esajyy5758fqsjyrz38jg5 --content-disposition --show-progress

!unzip -q src_clean.zip
!unzip -q src_noisy.zip

/content/train
clean.tar           100%[===================>]   8.95G  11.5MB/s    in 13m 13s 
noisy.tar           100%[===================>]   8.95G  15.2MB/s    in 12m 27s 
/content/test
src_clean.zip       100%[===================>]  34.24M  15.4MB/s    in 2.2s    
src_noisy.zip       100%[===================>]  38.62M  13.1MB/s    in 3.0s    


In [ ]:
!sudo apt-get -yqq install ffmpeg soundstretch

!pip install -q julius hydra_core==0.11.3 hydra-colorlog pesq pystoi protobuf==3.9.2

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libsoundtouch1:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libsoundtouch1_1.9.2-3_amd64.deb ...
Unpacking libsoundtouch1:amd64 (1.9.2-3) ...
Selecting previously unselected package soundstretch.
Preparing to unpack .../soundstretch_1.9.2-3_amd64.deb ...
Unpacking soundstretch (1.9.2-3) ...
Setting up libsoundtouch1:amd64 (1.9.2-3) ...
Setting up soundstretch (1.9.2-3) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers fo

In [ ]:
%cd /content

!wget https://github.com/YunyangZeng/TAPLoss/archive/refs/heads/master.zip
!unzip -q master.zip

%cd /content/TAPLoss-master/Demucs/denoiser
!pip install -r requirements_cuda.txt

/content
--2022-11-21 14:13:34--  https://github.com/YunyangZeng/TAPLoss/archive/refs/heads/master.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/YunyangZeng/TAPLoss/zip/refs/heads/master [following]
--2022-11-21 14:13:34--  https://codeload.github.com/YunyangZeng/TAPLoss/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [        <=>         ]  83.87M  17.1MB/s    in 8.9s    

2022-11-21 14:13:43 (9.43 MB/s) - ‘master.zip’ saved [87948764]

/content/TAPLoss-master/Demucs/denoiser
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [ ]:
!cat /content/TAPLoss-master/Demucs/denoiser/conf/config.yaml

defaults:
  - dset: debug
  - hydra/job_logging: colorlog
  - hydra/hydra_logging: colorlog

# Dataset related
sample_rate: 16000
segment: 10
stride: 2   # in seconds, how much to stride between training examples
pad: true   # if training sample is too short, pad it

# Dataset Augmentation
remix: false   # remix noise and clean
bandmask: 0.   # drop at most this fraction of freqs in mel scale
shift: 16000    # random shift, number of samples
shift_same: True   # shift noise and clean by the same amount
revecho: 0  # add reverb like augment

# Logging and printing, and does not impact training
num_prints: 5
device: cuda
num_workers: 16
verbose: 0
show: 0   # just show the model and its size and exit

# Checkpointing, by default automatically load last checkpoint
checkpoint: true
continue_from: 
      # Path the a checkpoint.th file to start from.
      # this is not used in the name of the experiment!
      # so use a dummy=something not to mixup experiments.
continue_best: false  # con

In [ ]:
%cd /content/TAPLoss-master/Demucs/denoiser

!python -m denoiser.audio /content/test/src_noisy > /content/test/noisy.json
!python -m denoiser.audio /content/test/src_clean > /content/test/clean.json

!python -m denoiser.audio /content/train/noisy > /content/train/noisy.json
!python -m denoiser.audio /content/train/clean > /content/train/clean.json

/content/TAPLoss-master/Demucs/denoiser


In [ ]:
dns_config = """
#!/bin/bash

dset:
  train: /content/train/
  valid: /content/test/
  test: /content/test/
  noisy_json:
  noisy_dir:
  matching: dns
#eval_every: 1
"""

with open("/content/TAPLoss-master/Demucs/denoiser/conf/dset/custom_dns.yaml", "w") as f:
    # Writing data to a file
    f.write(dns_config)

In [ ]:
# Line 33 use dns64
# Line 28 continue from is to use the checkpoint you have (remember to comment out Line 33)

In [ ]:
tap_config = """
#!/bin/bash

python train.py \
  dummy=waveform+1_pho_seg_ac_loss \
  continue_pretrained='dns64' \
  dset=custom_dns \
  acoustic_loss=True \
  acoustic_loss_only=False \
  phoneme_segmented=False \
  stft_loss=True \
  ac_loss_weight=1 \
  stft_loss_weight=0.3 \
  epochs=40 \
  num_workers=2 \
  batch_size=40 \
  ddp=0 $@
"""

with open("/content/TAPLoss-master/Demucs/denoiser/custom_tap_dns.sh", "w") as f:
    # Writing data to a file
    f.write(tap_config)

In [ ]:
run_config = """
#!/bin/bash

python train.py \
  dummy=waveform+1_pho_seg_ac_loss \
  continue_pretrained='dns64' \
  dset=custom_dns \
  acoustic_loss=False \
  acoustic_loss_only=False \
  phoneme_segmented=False \
  stft_loss=False \
  ac_loss_weight=1 \
  stft_loss_weight=0.3 \
  epochs=40 \
  num_workers=2 \
  batch_size=40 \
  ddp=0 $@
"""

with open("/content/TAPLoss-master/Demucs/denoiser/custom_base_dns.sh", "w") as f:
    # Writing data to a file
    f.write(run_config)

In [ ]:
!cat /content/TAPLoss-master/Demucs/denoiser/conf/dset/custom_base_dns.sh

cat: /content/TAPLoss-master/Demucs/denoiser/conf/dset/custom_base_dns.sh: No such file or directory


In [ ]:
import os

os.makedirs("/home/yunyangz/Documents/Demucs/with_acoustic_loss/LLD_Estimator_STFT/ckpts/")

In [ ]:
!wget -q https://cmu.box.com/shared/static/hv1xqf4m0iw86ffvtqrn9b4e2etg84nr --content-disposition --show-progress

!mv /content/TAPLoss-master/Demucs/denoiser/lld-estimation-model_12mse_14mae.pt /home/yunyangz/Documents/Demucs/with_acoustic_loss/LLD_Estimator_STFT/ckpts/lld-estimation-model_12mse_14mae.pt

lld-estimation-mode 100%[===================>]  64.08M  16.3MB/s    in 4.8s    


In [ ]:
!sh /content/TAPLoss-master/Demucs/denoiser/custom_base_dns.sh

[2022-11-21 14:14:09,634][__main__][INFO] - For logs, checkpoints and samples check /content/TAPLoss-master/Demucs/denoiser/outputs/exp_ac_loss_weight=1,acoustic_loss=False,acoustic_loss_only=False,batch_size=40,continue_pretrained=dns64,dset=custom_dns,dummy=waveform+1_pho_seg_ac_loss,epochs=40,phoneme_segmented=False,stft_loss=False,stft_loss_weight=0.3
learn rate:  0.0003
[2022-11-21 14:14:18,621][denoiser.solver][INFO] - Fine tuning from pre-trained model dns64
Downloading: "https://dl.fbaipublicfiles.com/adiyoss/denoiser/dns64-a7761ff99a7d5bb6.th" to /root/.cache/torch/hub/checkpoints/dns64-a7761ff99a7d5bb6.th
100% 128M/128M [00:16<00:00, 8.20MB/s]
2022-11-21 14:14:36.950634: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
[2022-11-21 14:14:38,692][numexpr.utils][INF